# 오디오 생성

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI
client = OpenAI()

### 모델로 부터 audio 출력

In [7]:
import warnings
warnings.filterwarnings('ignore')
import base64

Model = "gpt-4o-mini-audio-preview"

completion = client.chat.completions.create(
    model=Model,
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": "진도개는 주인에게 충성스러운가요?"
        }
    ]
)

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("output/dog.wav", "wb") as f:
    f.write(wav_bytes)

### Audio를 model에 입력하고 Text 와 Audio 를 동시에 출력

In [4]:
import base64
import requests

# Fetch the audio file and convert it to a base64 encoded string
url = "https://openaiassets.blob.core.windows.net/$web/API/docs/audio/alloy.wav"
response = requests.get(url)
response.raise_for_status()    # HTTP 에러 발생 시 예외 발생
wav_data = response.content
encoded_string = base64.b64encode(wav_data).decode('utf-8')

completion = client.chat.completions.create(
    model=Model,
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": [
                { 
                    "type": "text",
                    "text": "이 녹음파일에는 어떤 내용이 들어 있나요?"
                },
                {
                    "type": "input_audio",
                    "input_audio": {
                        "data": encoded_string,
                        "format": "wav"
                    }
                }
            ]
        },
    ]
)

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("output/audio_out.wav", "wb") as f:
    f.write(wav_bytes)

In [5]:
completion.choices[0].message.audio.transcript

'이 문장은 동양의 고대 역사와 지리학에서도 중요한 관측으로 알려져 있습니다. 해는 동쪽에서 뜨고 서쪽에서 지며, 이는 지구가 자전하기 때문입니다. 이러한 관찰은 고대 문명들의 천문학 연구의 기초가 되었으며, 시간을 측정하거나 방향을 정하는 데에도 사용되었습니다. 이러한 정보는 자연 현상을 이해하고 이용하는 데 있어 중요한 역할을 합니다.'

# Text to Audio
- 텍스트를 오디오로 변환
- 6개의 내장 음성 제공
( alloy, echo, fable, onyx, nova및 shimmer)

In [6]:
Model = "tts-1"

speech_file_path = "output/speech.mp3"

response = client.audio.speech.create(
  model=Model,
  voice="alloy",
  input="""
  한국의 자연은 울창한 산림, 아름다운 해안선, 그리고 계절마다 변화하는 아름다운 경관으로 유명합니다. 
  봄에는 벚꽃이, 가을에는 단풍이 화려하며, 전국에 흩어진 국립공원과 하천은 탐방객들에게 인기 있는 명소입니다.
  """
)

response.stream_to_file(speech_file_path)